In [1]:
!pip install pytorchvideo

You should consider upgrading via the '/home/chaos/.virtualenvs/dsenv/bin/python -m pip install --upgrade pip' command.


In [2]:
start= 0
end = 1

In [3]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys





class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

/home/chaos/.virtualenvs/dsenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60



train_inputs =[] #x
train_classes = [] #y


def transform_data(x, mean, std):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                
                Normalize((mean,), (std,)),

                CenterCrop([720,900]),
                Resize([480,600]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)

            
    video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))
#             video_data["video"] = video_data["video"]/255
            #print(video_data["video"].shape)
            
    std, mean = torch.std_mean(video_data["video"])
    std = std/255.0
    mean = mean/255.0
    video_data = transform_data( video_data, mean, std)

    return video_data["video"]

def load_dataloaders(batch_size,start,end):

    time_start = []
    time_end = []

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
            for i in range(len(time_start)-1):
                if time_end[i]- time_start[i+1]>400:
                    start = time_end[i]+50
                    end = time_end[i]+300
                    vid = lp_video(video,x[1], x[2])
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    
    vallen= int(len(val_test_set)*0.8)
    val_set, test_set = torch.utils.data.random_split(val_test_set, [vallen, len(val_test_set)-vallen ])
    valloader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)
    return trainloader, valloader, testloader 
        
    

In [19]:
with open('MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
for key in list(data.keys())[start:end]:
    filename = key
    print("file",filename)
    video = EncodedVideo.from_path("p04_n074.mp4")
    video_data = video.get_clip(1, 2)
    video_data['video'] = video_data['video'].permute(0,2,3,1)
    print(video_data["video"].shape)
    for m in torch.unbind(video_data['video'], dim=3):
        print(m.shape)

file p01_n002
torch.Size([3, 720, 1280, 25])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])
torch.Size([3, 720, 1280])


In [5]:


class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, growthRate=12, depth=10, reduction=0.5, nClasses=61, bottleneck=True):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(1, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
        out = F.log_softmax(self.fc(out))
        return out

In [6]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            pred = output.max(1, keepdim=True)[1]# get the index of the max log-probability
            
            num_samples += pred.shape[0]
            
#             print("num_samples:", num_samples)
            #print(torch.sum(pred==target))
            correct += (pred == target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))
    
def trainonval(model, device,validloader, optimizer, criterion, epochs):

    
    model.train()

    
    
    print("Training on Validation Set starts with lr",optimizer.param_groups[0]['lr'])
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(validloader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            pred = (output.data).max(1, keepdim=True)[1]# get the index of the max log-probability
            num_samples += pred.shape[0]
            correct += torch.sum(pred.data==target)
            
        
        #print(scheduler.get_last_lr())
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 90:
            
            print('Epoch: {} , Training Accuracy on Val set: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            test(model, device, testloader, mode = "Test after Training on validation set")
                
            return model, optimizer
            
    
        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        
    return model, optimizer

def train(model, device, train_loader,validloader,testloader, optimizer,scheduler,criterion, epochs):
    print("Train start")
    breakout = False
    model.half()
    model.cuda()

    optimizer.param_groups[0]['lr'] = 1e-2
    optimizer.param_groups[1]['lr'] = 1e-3
  

    model.train()

    
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            pred = output.max(1, keepdim=True)[1]# get the index of the max log-probability
            num_samples += pred.shape[0]
            correct += torch.sum(pred==target)
            
        #if optimizer.param_groups[0]['lr'] == 1e-2:    
        scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 93:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            test(model,device,validloader, mode = "Validating")
            test(model, device, testloader, mode = "Test before Training on validation set")
            model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
            test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")
            
            breakout = True
            break
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        if (((epoch+1)%5) == 0) :
            test(model,device,validloader)
            model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
    test(model, device, testloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")
    
    
        


In [7]:
n_classes = 60


model = nn.Sequential(OrderedDict([
    ('frontend',DenseNet()),
    ('fc', nn.Sequential(nn.Linear(1458, 60)))
]))





# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        {"params": model.fc.parameters(), "lr": 1e-2},
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
# state = torch.load("../input/sign-classification/model_optimizer1.pt")
# model.load_state_dict(state['model_state_dict'])
# model.half()
# model.cuda()
# optimizer.load_state_dict(state['optimizer_state_dict'])

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainloader, valloader, testloader = load_dataloaders(batch_size=4,start=start, end=end)#73 not included 


file p01_n002
file p06_n019
file p06_n007
file p01_n053
file p04_n074
file p05_n077
file p05_n027
file p05_n022
file p01_n110
file p06_n036


In [9]:
print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

Number of model parameters: 25887050


In [10]:
train(model,device,trainloader,valloader, testloader,optimizer,scheduler,criterion,50)

Train start
Epoch: 1 , Training Accuracy: 41/167 (25%) Training Loss: 0.978516
Epoch: 2 , Training Accuracy: 52/167 (31%) Training Loss: 0.937851
Epoch: 3 , Training Accuracy: 39/167 (23%) Training Loss: 0.909770
Epoch: 4 , Training Accuracy: 51/167 (31%) Training Loss: 0.889666
Epoch: 5 , Training Accuracy: 54/167 (32%) Training Loss: 0.876579
Validate Accuracy: 2/33 (6%)
Epoch: 6 , Training Accuracy: 64/167 (38%) Training Loss: 0.862685
Epoch: 7 , Training Accuracy: 48/167 (29%) Training Loss: 0.834628
Epoch: 8 , Training Accuracy: 69/167 (41%) Training Loss: 0.807108
Epoch: 9 , Training Accuracy: 79/167 (47%) Training Loss: 0.788256
Epoch: 10 , Training Accuracy: 75/167 (45%) Training Loss: 0.761169
Validate Accuracy: 12/33 (36%)
Epoch: 11 , Training Accuracy: 72/167 (43%) Training Loss: 0.755807
Epoch: 12 , Training Accuracy: 78/167 (47%) Training Loss: 0.736621
Epoch: 13 , Training Accuracy: 95/167 (57%) Training Loss: 0.704394
Epoch: 14 , Training Accuracy: 81/167 (49%) Training 

In [11]:
 #test(model, device, testloader)

In [12]:
#from 21 15 epochs